<a href="https://colab.research.google.com/github/victorgau/python_crawler/blob/master/4-3%20%E6%8A%93%E5%8F%96XML%E8%B3%87%E6%96%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 抓取XML資料

提示：

使用 BeautifulSoup 也可以剖析 XML 檔喔！

```python
soup = BeautifulSoup(r.text, 'xml')
```

參考資料：

* [Wikipedia - XML](https://zh.wikipedia.org/wiki/XML)
* [高雄市公共腳踏車即時租賃站資訊](https://data.kcg.gov.tw/dataset/public-bike-rental/resource/1440e8fb-a87c-4eef-bdc7-9f5bc6a0481a)

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://www.c-bike.com.tw/xml/stationlistopendata.aspx"

In [ ]:
r = requests.get(url)

In [ ]:
soup = BeautifulSoup(r.text, 'xml')

In [ ]:
stations = soup.select('Station')

In [ ]:
stations[0]

In [ ]:
import folium
import geocoder

In [ ]:
# 使用 geocoder 取得特定住址的 GPS 座標
location = geocoder.osm('高雄市').latlng

In [ ]:
m = folium.Map(location=location, zoom_start=16)

In [ ]:
for station in stations:
    try:
        lat = float(station.select('StationLat')[0].text)
        lon = float(station.select('StationLon')[0].text)
        name = station.select('StationName')[0].text
        num1 = station.select('StationNums1')[0].text
        num2 = station.select('StationNums2')[0].text
        folium.Marker([lat, lon], popup="<div style='white-space:nowrap;text-align:center;'>{}<br>(可借：{}, 可停：{})</div>".format(name, num1, num2)).add_to(m)
    except Exception as e:
        print(e.args)

In [ ]:
m